## Data pre-processing

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/interim/nhanes_data_interim.csv")

### Convert categorical variables represented by float to int

In [3]:
df["diabetes"] = df["diabetes"].astype(int)
df["current_smoking_freq"] = df["current_smoking_freq"].astype(int)

### Transform highly-skewed features

In [4]:
# Log transformation on right-skewed features
cols_to_transform = ["hba1c_percentage", "fpg_mmol_L", "hdl_mmol_L"]

for col in cols_to_transform:
    df[col + "_log"] = np.log(df[col])

# Drop original columns
df = df.drop(cols_to_transform, axis=1)

### Identify outliers

In [5]:
def three_sd_range(series):
    mean = series.mean()
    sd = series.std()
    lower_bound = mean - 3 * sd
    upper_bound = mean + 3 * sd
    
    return (lower_bound, upper_bound)

In [6]:
for col_name in ["bmi", "age", "hba1c_percentage_log", "fpg_mmol_L_log", "hdl_mmol_L_log"]:
    lower_bound, upper_bound = three_sd_range(df[col_name])
    outlier_present = (df[col_name].min() < lower_bound or df[col_name].max() > upper_bound)
    print(f"{col_name} has at least one outlier: {outlier_present}")
    
    if outlier_present:
        curr_length = df[col_name].count()
        df = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
        new_length = df[col_name].count()
        print(f"{curr_length - new_length} row has been removed")
    
    print()

bmi has at least one outlier: True
11 row has been removed

age has at least one outlier: False

hba1c_percentage_log has at least one outlier: True
31 row has been removed

fpg_mmol_L_log has at least one outlier: True
27 row has been removed

hdl_mmol_L_log has at least one outlier: True
6 row has been removed



### Removing features that provide no predictive information

In [7]:
col_one_uniq = []
for col in df.columns:
    if df[col].nunique() <= 1:
        col_one_uniq.append(col)
        
df = df.drop(col_one_uniq, axis=1)
print(f"{len(col_one_uniq)} column(s) has been removed")

1 column(s) has been removed


## Feature Engineering

In [8]:
# df["bmi_age_interaction"] = df["bmi"] * df["age"] / 100
# df["glucose_hba1c_interaction"] = df["fpg_mmol_L_log"] * df["hba1c_percentage_log"]
# df["bp_age_interaction"] = df["high_blood_pressure"] * df["age"]

In [9]:
# df.info()

## Label encoding

In [10]:
# Encode target variable
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["diabetes"] = label_encoder.fit_transform(df["diabetes"])

In [11]:
# Resultant DataFrame after label encoding
df.sample(5)

,bmi,high_blood_pressure,gender,age,diabetes,current_smoking_freq,hba1c_percentage_log,fpg_mmol_L_log,hdl_mmol_L_log
634,23.3,1,1,58,1,3,1.648659,1.640937,0.254642
864,29.3,0,0,51,1,3,1.629241,1.713798,0.336472
543,37.2,0,0,53,1,1,2.091864,2.057963,0.239017
184,24.4,1,1,66,2,3,1.931521,1.896119,0.058269
398,30.7,0,0,29,1,3,1.609438,1.652497,0.173953


In [12]:
df["diabetes"].value_counts()

diabetes
1    928
0    155
2     51
Name: count, dtype: int64

In [13]:
# Save the pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed_label_encoded.csv", index=False)

## Categorical encoding

In [14]:
# Map multiclass values to more readable strings to prepare for one-hot encoding
df["diabetes"] = df["diabetes"].map({0: "yes", 1: "no", 2: "borderline"})
df["current_smoking_freq"] = df["current_smoking_freq"].map({1: "every_day", 2: "some_days", 3: "none"})

In [15]:
# One-hot encode the multiclass columns "diabetes" and "current_smoking_frequency"
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, dtype=int)
one_hot_encoded = encoder.fit_transform(df[["diabetes", "current_smoking_freq"]])

encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(["diabetes", "current_smoking_freq"]))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original multiclass columns after encoding
df = df.drop(columns=["diabetes", "current_smoking_freq"], axis=1)

In [16]:
# Resultant DataFrame after one-hot encoding
df.sample(5)

,bmi,high_blood_pressure,gender,age,hba1c_percentage_log,fpg_mmol_L_log,hdl_mmol_L_log,diabetes_borderline,diabetes_no,diabetes_yes,current_smoking_freq_every_day,current_smoking_freq_none,current_smoking_freq_some_days
464,27.2,0.0,1.0,47.0,1.629241,1.704748,0.029559,0.0,0.0,1.0,1.0,0.0,0.0
531,26.6,1.0,0.0,65.0,1.757858,1.662030,0.703098,0.0,1.0,0.0,1.0,0.0,0.0
967,38.6,0.0,1.0,51.0,1.686399,1.791759,0.058269,0.0,1.0,0.0,0.0,1.0,0.0
795,30.2,1.0,1.0,59.0,1.648659,1.693779,0.371564,0.0,1.0,0.0,0.0,1.0,0.0
1208,27.9,0.0,0.0,69.0,2.091864,2.183802,-0.127833,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Save the one-hot encoded pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed_one_hot_encoded.csv", index=False)